In [33]:
import pandas as pd
import hashlib
from datetime import datetime

In [34]:
# Function to generate a hash from text
def generate_hash(text):
    return hashlib.sha256(text.encode()).hexdigest()


t = "Hello World"
a = generate_hash(t)
print(t)
print(a)
print(datetime.now())

Hello World
a591a6d40bf420404a011733cfb7b190d62c65bf0bcda32b57b277d9ad9f146e
2024-12-08 20:08:10.586861


In [35]:
DF = pd.DataFrame({
    'text': [t],
    'hash': [a],
    'date': [datetime.now()]
})

In [36]:
mask = DF['hash'] == a
df = DF[mask]

df_not = DF[~mask]

In [37]:
DF.to_pickle('/app/data.pkl')

In [1]:
import os
import re
import time
import requests
from urllib.parse import urljoin, urlparse
from collections import deque
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import hashlib
from datetime import datetime



class Scraper:
    def __init__(self, URLs, output_dir='/app/BFS', max_depth=4,frame=None):
        self.URLs = URLs
        self.max_depth = max_depth
        self.visited_urls = set()
        self.frame = frame
        self.output_dir = output_dir
        self.check_dataframe()

    # Function to generate a hash from text
    def generate_hash(self,text):
        return hashlib.sha256(text.encode()).hexdigest()

    def check_dataframe(self):
        if self.frame is None:
            self.frame = pd.DataFrame(columns=['url', 'text_hash','date_accessed'])
        else:
            self.frame = pd.read_pickle(self.frame)

    def save_dataframe(self):
        self.frame.to_pickle(os.path.join(self.output_dir,'dataframe.pkl'))

    def url_to_filename(self,url):
        filename = re.sub(r'^(http|https)://', '', url)
        filename = filename.replace('/', '_')
        filename = re.sub(r'[^a-zA-Z0-9\-_]', '_', filename)
        max_length = 255
        return filename[:max_length]


    def reset_visited_urls(self):
        self.visited_urls = set()

    def scrape_text_bfs(self,start_url, base_url, max_depth=4):
        self.reset_visited_urls()
        
        queue = deque([(start_url, 0)])  # Queue stores tuples of (url, current_depth)

        while queue and len(self.visited_urls) < 5:
            url, depth = queue.popleft()
            print(f"Scraping {url} at depth {depth}")
            
            # Check if the URL has already been visited or if it exceeds max depth
            if url in self.visited_urls or depth > max_depth:
                continue
            
            mask = self.frame['url'] == url
            entry = self.frame[mask]
            
            try:
                date_accessed = entry['date_accessed'].values[0]
            except:
                date_accessed = None

            # Mark the URL as visited
            self.visited_urls.add(url)

            try:
                response = requests.get(url)
                response.raise_for_status()
            except requests.exceptions.RequestException as e:
                print(f"Failed to retrieve {url}: {e}")
                continue

            soup = BeautifulSoup(response.text, 'html.parser')
            if (mask.sum() == 1 and pd.Timestamp.now()-date_accessed >= pd.Timedelta(weeks=1)) or mask.sum() == 0:
                # Extract and save text
                text = soup.get_text(separator=' ', strip=True)
                text = f"URL:{url}\n{text}"

                try:
                    text_hash = entry['text_hash'].values[0]
                except:
                    text_hash = None

                file_path = os.path.join(self.output_dir, self.url_to_filename(url) + '.txt')

                #See if the stored hash is the same as the hash for current text
                if text_hash == self.generate_hash(text):
                    pass
                elif mask.sum() == 0:
                    
                    with open(file_path, 'w') as f:
                        f.write(text)

                    # Add the URL to the dataframe
                    self.frame = pd.concat([self.frame , pd.DataFrame({'url': [url], 'text_hash': [self.generate_hash(text)],'date_accessed':[datetime.now()],'file_path': [file_path]})], ignore_index=True)
                
                elif mask.sum() == 1:
                    #Delete the corresponding file
                    os.remove(entry['file_path'].values[0])
                    
                    #Delete the entry in the dataframe
                    self.frame = self.frame[~mask]

                    
                    with open(file_path, 'w') as f:
                        f.write(text)

                    # Add the URL to the dataframe
                    self.frame = pd.concat([self.frame , pd.DataFrame({'url': [url], 'text_hash': [self.generate_hash(text)],'date_accessed':[datetime.now()],'file_path': [file_path]})], ignore_index=True)
            
            
            # If the current depth is less than max_depth, find and add links to the queue
            if depth < max_depth:
                for link in soup.find_all('a', href=True):
                    href = link['href']
                    next_url = urljoin(base_url, href)
                    if urlparse(next_url).netloc == urlparse(base_url).netloc and next_url not in self.visited_urls:
                        queue.append((next_url, depth + 1))
                        time.sleep(0.5)  # Sleep for 500ms to avoid hammering the server
        self.save_dataframe()

    def breadth_scrape(self):
        for start_url in tqdm(self.URLs):
            self.scrape_text_bfs(start_url, start_url, max_depth=self.max_depth)

In [2]:
scrape = Scraper(['https://health.uoregon.edu/'],'/app/obj_test/',frame='/app/obj_test/dataframe.pkl')

In [3]:
scrape.breadth_scrape()

  0%|          | 0/1 [00:00<?, ?it/s]

Scraping https://health.uoregon.edu/ at depth 0
Scraping https://health.uoregon.edu/#main-content at depth 1
Scraping https://health.uoregon.edu/search at depth 1
Scraping https://health.uoregon.edu/search at depth 1
Scraping https://health.uoregon.edu/medical-care at depth 1
Scraping https://health.uoregon.edu/primary-care at depth 1


100%|██████████| 1/1 [05:12<00:00, 312.64s/it]


In [4]:
import os
import re
import time
import requests
from urllib.parse import urljoin, urlparse
from collections import deque
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import hashlib
from datetime import datetime
from openai import OpenAI
import yaml


class Builder:
    def __init__(self, directory, instructions,VS_name):
        self.client = OpenAI()
        self.directory = directory
        self.txt_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.txt')]
        self.list_of_dicts = self.initialize_list_of_dicts()
        self.file_ids = []
        self.instructions = instructions
        self.vector_store = None
        self.VS_name = VS_name

    def initialize_list_of_dicts(self):
        size = len(self.txt_files)
        keys = ['filename', 'file_id', 'vectorstore_id', 'assistant_id']
        list_of_dicts = [{key: None for key in keys} for _ in range(size)]
        for i, d in enumerate(list_of_dicts):
            d["filename"] = self.txt_files[i]
        return list_of_dicts

    def create_files(self):
        for txt_file in self.txt_files:
            try:
                with open(txt_file, 'rb') as file:
                    upload_file = self.client.files.create(
                        file=file,
                        purpose='assistants'
                    )
                    self.file_ids.append(upload_file.id)
            except Exception as e:
                print(f"Error uploading file {txt_file}: {e}")

        for i, d in enumerate(self.list_of_dicts):
            d["file_id"] = self.file_ids[i]

    def create_vector_store(self):
        try:
            vector_store = self.client.beta.vector_stores.create(name=self.VS_name)
            self.vector_store = vector_store
            batch = self.client.beta.vector_stores.file_batches.create_and_poll(
                vector_store_id=vector_store.id,
                file_ids=self.file_ids
            )
            for d in self.list_of_dicts:
                d["vectorstore_id"] = vector_store.id
        except Exception as e:
            print(f"Error creating vector store: {e}")

    def create_assistant(self):
        try:
            assistant = self.client.beta.assistants.create(
                name="Puddles_v2",
                instructions=self.instructions,
                model="gpt-4o-mini",
                tools=[{"type": "file_search"}],
                tool_resources={
                    "file_search": {
                        "vector_store_ids": [self.vector_store.id]
                    }
                }
            )
            for d in self.list_of_dicts:
                d["assistant_id"] = assistant.id
            return assistant
        except Exception as e:
            print(f"Error creating assistant: {e}")
            return None

    def save_to_csv(self, filepath):
        try:
            df = pd.DataFrame(self.list_of_dicts)
            df.to_csv(filepath, index=False)
        except Exception as e:
            print(f"Error saving to CSV: {e}")

    def initialize_assistant(self):
        self.create_files()
        self.create_vector_store()
        if self.vector_store is not None:
            self.create_assistant()
            self.save_to_csv(f'{self.directory}/assistant_files.csv') #Need to get rid from the hardcoding
        return self.list_of_dicts[0]['assistant_id']

    def delete_assistant(self):
        try:
            # Delete all files
            list_files = self.client.files.list(purpose='assistants')
            for file in list_files:
                self.client.files.delete(file.id)
                print(f"Deleted file with id: {file.id}")

            # Delete the assistant
            if self.list_of_dicts and self.list_of_dicts[0]['assistant_id']:
                assistant_id = self.list_of_dicts[0]['assistant_id']
                self.client.beta.assistants.delete(assistant_id)
                print(f"Deleted assistant with id: {assistant_id}")

            # Delete the vector store
            if self.list_of_dicts and self.list_of_dicts[0]['vectorstore_id']:
                vectorstore_id = self.list_of_dicts[0]['vectorstore_id']
                self.client.beta.vector_stores.delete(vectorstore_id)
                print(f"Deleted vector store with id: {vectorstore_id}")
        except Exception as e:
            print(f"Error deleting resources: {e}")

In [5]:
with open('/app/puddles_prompt.yml') as f:
    context = yaml.load(f, Loader=yaml.FullLoader)['context']

builder = Builder('/app/Store/', context,'Puddles_VS')

In [4]:
from openai import OpenAI
from openai.types.beta.assistant_stream_event import ThreadRunStepCompleted

client = OpenAI()
assistant = client.beta.assistants.retrieve(assistant_id="asst_WvxFY77dyhKaV6ei809wTfye")
thread = client.beta.threads.create()
user_query = "How do I become involved in the LGBTQ community?"
# Add user query to the thread
client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_query
    )
stream = client.beta.threads.runs.create(
            thread_id=thread.id,
            assistant_id="asst_WvxFY77dyhKaV6ei809wTfye",
            instructions = "If the user has a specific question, you should run a file search for what is specifically available at the University of Oregon",
            stream=True,
            tool_choice='required'
            )

for event in stream:
# There are various types of streaming events
# See here: https://platform.openai.com/docs/api-reference/assistants-streaming/events
    print('new')
    # Here, we only consider if there's a delta text
    if isinstance(event, ThreadRunStepCompleted):
        print(event.data.step_details)
       


new
new
new
new
new
new
new
ToolCallsStepDetails(tool_calls=[FileSearchToolCall(id='call_XbQILel3JsKWI1nBRSWscfg9', file_search=FileSearch(ranking_options=FileSearchRankingOptions(ranker='default_2024_08_21', score_threshold=0.0), results=[FileSearchResult(file_id='file-W9gaJ8VbTJzSKggGt16c63', file_name='counseling_uoregon_edu_groups.txt', score=0.6717550245021959, content=None), FileSearchResult(file_id='file-Kupqr3C9ZUcACBep4z35Mk', file_name='engage_uoregon_edu_thehub_.txt', score=0.644210735781246, content=None), FileSearchResult(file_id='file-RwjZ6py3NMsu8CV5wgqsmA', file_name='advising_uoregon_edu_quick-questions_standing_aw.txt', score=0.631530340586583, content=None), FileSearchResult(file_id='file-8qdTdcfiTmeMA8Ku2KeTQy', file_name='advising_uoregon_edu_quick-questions_other_dqconsiderations.txt', score=0.6276619480665055, content=None), FileSearchResult(file_id='file-AawP6cymphvXvw4UwspKsf', file_name='advising_uoregon_edu_quick-questions_first.txt', score=0.6261712105894829

In [ ]:
event.data.step_details.tool_calls[0].file_search.results

def get_files_used(message_id,thread_id):
    message = client.beta.threads.messages.retrieve(
        message_id=message_id,
        thread_id=thread_id,
    )
    annotations = message.content[0].text.annotations

    files_used = []
    for annotation in annotations:
        file = annotation.file_citation.file_id
        files_used = files_used.append(file) 
    return files_used

files_used = get_files_used("msg_oNOIt14F23h2LAdxVuoiwoSS","thread_G0dEvZuMiLzkvy0eASH5piqm")
print(files_used)

In [6]:
from openai.types.beta.assistant_stream_event import ThreadMessageDelta
from openai.types.beta.threads.text_delta_block import TextDeltaBlock 
# Iterate through the stream 

output = ""
for event in stream:
# There are various types of streaming events
# See here: https://platform.openai.com/docs/api-reference/assistants-streaming/events

    # Here, we only consider if there's a delta text
    if isinstance(event, ThreadMessageDelta):
        if isinstance(event.data.delta.content[0], TextDeltaBlock):
            # empty the container

            # add the new text
            output = f"{output} {event.data.delta.content[0].text.value}"
            print(event.data.delta.content[0].text.value)


To
 become
 involved
 in
 the
 LGBTQ
 community
,
 especially
 at
 the
 University
 of
 Oregon
,
 you
 have
 several
 options
 and
 resources
 available
:


1
.
 **
L
GBT
Q
+
 Support
 Groups
**
:
 Join
 support
 groups
 like
 the
 LGBTQ
+
 Support
 Group
,
 which
 is
 held
 on
 Wednesdays
 from
 
2
 PM
 to
 
4
 PM
.
 This
 is
 a
 safe
 space
 for
 individuals
 who
 identify
 as
 LGBTQ
+
 to
 discuss
 their
 experiences
 without
 judgment
.
 You
 can
 get
 more
 information
 by
 calling
 
541
-
346
-
322
7
【4:0†source】
.


2
.
 **
Student
 Organizations
**
:
 The
 University
 of
 Oregon
 hosts
 various
 student
 organizations
 focused
 on
 LGBTQ
+
 identities
.
 For
 instance
,
 the
 Lesbian
,
 Gay
,
 Bis
exual
,
 Trans
,
 Que
er
,
 A
sexual
,
 Arom
antic
,
 and
 Allied
 Alliance
 (
L
GBT
QA
3
),
 and
 the
 Que
er
,
 Trans
,
 and
 Inter
sex
 Students
 of
 Color
 (
QT
IS
OC
)
 are
 two
 examples
.
 Getting
 involved
 in
 these
 groups
 can
 foster
 community
 connections
【4:6†source】
.


In [27]:
client = OpenAI()

a = client.beta.assistants.list()
for asst in a:
        # keys via __dict__:
        #     id='asst_7KIgsjdEjqSfsguBwI7UIXy7'
        #     created_at=1721336916
        #     description=None
        #     instructions=None
        #     metadata={}
        #     model='gpt-3.5-turbo'
        #     name='Pretty Name'object='assistant'
        #     tools=[FileSearchTool(type='file_search', file_search=None)]
        #     response_format='auto'
        #     temperature=1.0
        #     tool_resources=ToolResources(code_interpreter=None
        #     file_search=ToolResourcesFileSearch(vector_store_ids=[]))top_p=1.0
    asst_dict = asst.to_dict()
    
    if asst_dict["name"] == 'Ask Puddles':
        print(asst_dict["name"])
        print(asst_dict["id"])
 

Ask Puddles
asst_E80fqsP0WMJrkYuBs15WbF7x


In [24]:
assistant = client.beta.assistants.retrieve(name="Ask Puddles")

TypeError: retrieve() got an unexpected keyword argument 'name'

In [18]:
c = asst.to_dict()
print(c['name'])

Ask Puddles


In [19]:
asst.to_dict()

{'id': 'asst_E80fqsP0WMJrkYuBs15WbF7x',
 'created_at': 1721856497,
 'description': None,
 'instructions': 'You are an agent that helps connect students with Wellness Resources at the University of Oregon.  Your job is to help students find the correct resource or resources for their given inquire.  You are kind, empathetic, and empowering!   Provide links to relevant webpages as often as possible.\n\nYour responses should start with a brief restatement of the user query. Next, you should provide a summary of available resources and relevant information. Then you should close by providing web links where more information can be found, if there are any provided.  \n\nNote that sometimes students will need to talk about sensitive subjects.  You must be compassionate, caring, and non-judgemental.',
 'metadata': {},
 'model': 'gpt-4o-mini',
 'name': 'Ask Puddles',
 'object': 'assistant',
 'tools': [{'type': 'file_search',
   'file_search': {'max_num_results': 30,
    'ranking_options': {'sc